# Балансировка классов в задачах классификации: сравнение методов (undersampling, oversampling, SMOTE)
## 1. Введение
В задачах бинарной или многоклассовой классификации часто возникает проблема дисбаланса классов — когда один класс (обычно называемый «миноритарным») представлен значительно меньшим количеством примеров, чем другой («мажоритарный»). Это приводит к тому, что модели машинного обучения склонны игнорировать миноритарный класс, максимизируя общую точность за счёт низкой чувствительности (recall).

Цель данной статьи — сравнить три популярных подхода к решению проблемы дисбаланса:

- Undersampling (уменьшение мажоритарного класса),
- Oversampling (увеличение миноритарного класса путём дублирования),
- SMOTE (синтетическая генерация примеров миноритарного класса).

Для сравнения будет использован реальный датасет с выраженным дисбалансом, и оценка эффективности методов будет проведена по метрикам, адекватным для несбалансированных данных.
## 2. Описание датасета
Датасет **Machine Predictive Maintenance Classification** моделирует ситуацию из промышленного производства или умного завода, где оборудование (например, станки, двигатели, роботизированные манипуляторы) работает непрерывно и генерирует данные о своём состоянии.

Основная цель — предиктивное (прогнозирующее) техническое обслуживание (Predictive Maintenance):

«Можно ли заранее предсказать, какой именно тип отказа произойдёт с машиной, чтобы вовремя остановить её, заменить деталь или отрегулировать параметры — и избежать простоев, аварий или брака?» 

В отличие от реактивного обслуживания (чиним, когда сломалось), планового обслуживания (меняем детали по графику, даже если они ещё работают), предиктивное обслуживание экономит деньги, повышает безопасность и эффективность.

**Признаки:**
1. UDI (числовой) - технический ID строки;
2. Product ID (строка) - уникальный ID продукта;
3. Type (категориальная) - тип оборудования/продукта (L, M, H — возможно, означает Low/Medium/High качество);
4. Air temperature [K] (числовой) - температура воздуха на входе (в Кельвинах);
5. Process temperature [K] (числовой) - температура в процессе обработки;
6. Rotational speed [rpm] (числовой) - скорость вращения (оборотов в минуту);
7. Torque [Nm] (числовой) - крутящий момент (ньютон-метры);
8. Tool wear [min] (числовой) - время износа инструмента (в минутах).

**Целевые переменные:**
1. Target (бинарный) - отказ оборудования (1), нет отказа (0);
2. FailureType (категориальная) - тип отказа оборудования (или его отсутствие).

**Классы целевой переменной:**
1. No Failure - отказа нет (количество: 9652)
2. Heat Dissipation Failure - отказ отвода тепла или перегрев (количество: 112)
3. Power Failure - сбой питания (количество: 95)
4. Overstrain Failure - механическая перегрузка (количество: 78)
5. Tool Wear Failure - отказ из-за износа инструмента (количество: 45)
6. Random Failures - случайные отказы (количество: 18)

Это крайне несбалансированный многоклассовый датасет — идеален для исследования методов балансировки.

**Подготовка данных:**
1. Удалим два столбца "UDI", "Product ID", они нам не понадобятся
2. 

In [4]:
import pandas as pd

df = pd.read_csv('predictive_maintenance.csv')
df = df.drop(columns=['UDI', 'Product ID', 'Target'])

df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type
0,M,298.1,308.6,1551,42.8,0,No Failure
1,L,298.2,308.7,1408,46.3,3,No Failure
2,L,298.1,308.5,1498,49.4,5,No Failure
3,L,298.2,308.6,1433,39.5,7,No Failure
4,L,298.2,308.7,1408,40.0,9,No Failure
